## Filtering Data

In [1]:
import pandas as pd
# Removing the first row "Notes offered by Prospectus" 
loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1)
# Removing all columns containing more than 50% missing values
half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
# Removing the desc column which contains a long text explanation for each loan
loans_2007 = loans_2007.drop(['desc'],axis=1)
# Saving the dataset as loans_2007
loans_2007.to_csv('loans_2007.csv', index=False)

C:\Users\ParsaG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
loans_2007.drop_duplicates()
# The first row of loans_2007
print(loans_2007.iloc[0])

loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                    

In [3]:
# Number of columns
print(loans_2007.shape[1])

53


In [4]:
# Removing some of the columns that are leaking future information or are not useful
loans_2007 = loans_2007.drop(["funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

In [5]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

In [6]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)

In [7]:
# The first row of loans_2007
print(loans_2007.iloc[0])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               F

In [8]:
# Number of columns
print(loans_2007.shape[1])

35


In [9]:
# Exploring the target column
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64


In [10]:
# Removing all rows from loans_2007 that contain values other than Fully Paid or Charged Off for the loan_status column.
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

In [11]:
# transforming the values to 0 and 1
status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

In [12]:
# Removing the columns with only 1 unique value after removing the null values
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens', 'hardship_flag', 'disbursement_method']


## Cleaning Data

### Missing Data

In [13]:
null_counts = loans_2007.isnull().sum()
print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
debt_settlement_flag      0
dtype: int64


In [14]:
# Removing the entire column with more than 1% null values
loans = loans_2007.drop("pub_rec_bankruptcies", axis=1)

In [15]:
# Removing the rows with any null value
loans = loans.dropna(axis=0)

In [16]:
print(loans.dtypes.value_counts())

object     12
float64    10
int64       1
dtype: int64


### Object Columns

In [17]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                      36 months
int_rate                     10.65%
emp_length                10+ years
home_ownership                 RENT
verification_status        Verified
purpose                 credit_card
title                      Computer
addr_state                       AZ
earliest_cr_line           Jan-1985
revol_util                    83.7%
last_credit_pull_d         Feb-2018
debt_settlement_flag              N
Name: 0, dtype: object


#### Categorical Columns

In [18]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())

RENT        18881
MORTGAGE    17688
OWN          3056
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16890
Verified           12833
Source Verified    10001
Name: verification_status, dtype: int64
10+ years    8897
< 1 year     4576
2 years      4389
3 years      4094
4 years      3435
5 years      3279
1 year       3240
6 years      2227
7 years      1771
8 years      1483
9 years      1259
n/a          1074
Name: emp_length, dtype: int64
 36 months    29041
 60 months    10683
Name: term, dtype: int64
CA    7095
NY    3815
FL    2869
TX    2729
NJ    1850
IL    1524
PA    1515
VA    1407
GA    1399
MA    1343
OH    1221
MD    1053
AZ     878
WA     841
CO     791
NC     788
CT     754
MI     722
MO     685
MN     613
NV     497
SC     472
WI     459
AL     451
OR     450
LA     436
KY     327
OK     299
KS     271
UT     259
AR     245
DC     212
RI     199
NM     190
WV     177
HI     173
NH     172
DE     113
MT      85
WY      83
AK      

In [19]:
print(loans["purpose"].value_counts())
print(loans["title"].value_counts())

debt_consolidation    18661
credit_card            5134
other                  3985
home_improvement       2980
major_purchase         2182
small_business         1827
car                    1549
wedding                 947
medical                 693
moving                  581
house                   382
vacation                380
educational             320
renewable_energy        103
Name: purpose, dtype: int64
Debt Consolidation                            2189
Debt Consolidation Loan                       1732
Personal Loan                                  661
Consolidation                                  516
debt consolidation                             508
Home Improvement                               357
Credit Card Consolidation                      357
Debt consolidation                             334
Small Business Loan                            329
Credit Card Loan                               319
Personal                                       309
Consolidation Loan 

In [20]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title", "debt_settlement_flag"], axis=1)
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

#### Encoding categorial values as dummy variable

In [21]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns]).astype("float")
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

## ML

In [22]:
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39724 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              39724 non-null float64
int_rate                               39724 non-null float64
installment                            39724 non-null float64
emp_length                             39724 non-null int64
annual_inc                             39724 non-null float64
loan_status                            39724 non-null int64
dti                                    39724 non-null float64
delinq_2yrs                            39724 non-null float64
inq_last_6mths                         39724 non-null float64
open_acc                               39724 non-null float64
pub_rec                                39724 non-null float64
revol_bal                              39724 non-null float64
revol_util                             39724 non-null float64
total_acc                              39724 non-null float64
home_ownership_MORTGAGE    

### Error metric
Our objective is to minimize risk by avoiding false positives as much as possible. Missing out on opportunities (false negatives) will be more tolarable than funding a risky loan (false positives).

### Logistic Regression

In [23]:
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, KFold
penalty = {
    0: 10,
    1: 1
}

In [25]:
lr = LogisticRegression(class_weight=penalty)

In [26]:
kf = KFold(features.shape[0], random_state=1)

In [27]:
predictions = cross_val_predict(lr, features, target, cv=kf)

In [ ]:
predictions = pd.Series(predictions)

In [ ]:
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)